# Almacenamiento en caché
El almacenamiento en caché (caching) de RDD en Spark es una técnica que se utiliza para mejorar el rendimiento de las aplicaciones de Spark que realizan múltiples operaciones sobre los mismos RDDs.

Cuando se realiza una operación sobre un RDD en Spark, como una transformación o una acción, Spark calcula el resultado y lo almacena en la memoria del cluster de Spark. Si posteriormente se realiza otra operación sobre el mismo RDD, Spark tiene que volver a calcular el resultado desde cero. Esto puede ser muy costoso en términos de tiempo de cómputo y puede reducir el rendimiento de la aplicación.

El almacenamiento en caché de RDD resuelve este problema al permitir que los resultados intermedios de un RDD se almacenen en memoria y se reutilicen en operaciones posteriores. Esto se logra mediante la llamada a la función cache() o persist() en un RDD. La función cache() almacena el RDD en memoria, mientras que la función persist() permite al usuario especificar un nivel de almacenamiento en disco o en memoria.

Cuando se almacena en caché un RDD, los resultados intermedios se almacenan en la memoria del cluster de Spark y se reutilizan en operaciones posteriores en lugar de tener que recalcularlos desde cero. Esto puede mejorar significativamente el rendimiento de las aplicaciones de Spark que realizan múltiples operaciones sobre los mismos RDDs.

Es importante tener en cuenta que la caché de RDDs puede ocupar una gran cantidad de memoria del cluster de Spark, por lo que se recomienda utilizarla con precaución y liberar la memoria caché cuando ya no se necesite, utilizando la función unpersist(). Además, la caché de RDDs puede ser especialmente útil cuando se realizan múltiples operaciones sobre un RDD grande, ya que permite reutilizar los resultados intermedios en lugar de volver a calcularlos cada vez.

## Niveles de almacenamiento
**MEMORY_ONLY**: Almacena el RDD en la memoria del cluster de Spark. Si el RDD no cabe en la memoria, algunas particiones no se almacenarán y tendrán que ser recalculadas cuando sea necesario. Es el nivel de almacenamiento por defecto.

**MEMORY_AND_DISK**: Almacena el RDD en la memoria del cluster de Spark y en el disco. Si el RDD no cabe en la memoria, algunas particiones se almacenarán en el disco en lugar de la memoria.

**DISK_ONLY**: Almacena el RDD en el disco del cluster de Spark.

**MEMORY_ONLY_2, MEMORY_AND_DISK_2, etc.**: Almacena el RDD en la memoria y/o en el disco del cluster de Spark, replicando cada partición en dos nodos.

In [15]:
import findspark

findspark.init()

from pyspark.sql import SparkSession
from pyspark import SparkContext
from pyspark.storagelevel import StorageLevel

spark = SparkSession.builder.master("local[*]").getOrCreate()

sc: SparkContext = spark.sparkContext

In [16]:
rdd = sc.parallelize(range(100000000))

In [17]:
rdd.persist(StorageLevel.MEMORY_ONLY)

PythonRDD[5] at RDD at PythonRDD.scala:53

In [18]:
# Es necesario unpersistir el RDD cuando se quiera cambiar el nivel de almacenamiento, o arrojará un error
rdd.unpersist()
rdd.persist(StorageLevel.MEMORY_AND_DISK)

PythonRDD[5] at RDD at PythonRDD.scala:53

In [19]:
rdd.unpersist()
rdd.cache()  # Equivalente a rdd.persist(StorageLevel.MEMORY_ONLY)

PythonRDD[5] at RDD at PythonRDD.scala:53

# Particionado de RDD
El particionado de RDD en PySpark se refiere a la forma en que se divide un RDD en partes más pequeñas llamadas particiones. Las particiones son la unidad básica de procesamiento en PySpark y pueden ser procesadas de forma independiente en diferentes nodos de un cluster de Spark.

La forma en que un RDD se particiona tiene un gran impacto en el rendimiento de las aplicaciones de PySpark. Si el RDD está mal particionado, es decir, si las particiones son demasiado grandes o demasiado pequeñas, puede haber un desequilibrio en la carga de trabajo entre los nodos del cluster de Spark, lo que puede afectar el rendimiento de la aplicación.

En PySpark, existen dos tipos de particionado de RDD: el particionado por defecto y el particionado personalizado.

El particionado por defecto se realiza automáticamente cuando se crea un RDD a partir de datos, como un archivo de texto o una tabla de base de datos. En este caso, PySpark utiliza un particionado predeterminado, que se basa en el número de núcleos de CPU en el cluster de Spark.

El particionado personalizado, por otro lado, permite al usuario controlar la forma en que un RDD se particiona. Para particionar un RDD de forma personalizada, se puede utilizar la función repartition() o coalesce() de PySpark.

Es importante tener en cuenta que la partición de RDD debe ser cuidadosamente seleccionada para optimizar el rendimiento de la aplicación. Un número excesivo de particiones puede aumentar el tiempo de latencia y reducir el rendimiento, mientras que un número insuficiente de particiones puede disminuir la capacidad de procesamiento paralelo y aumentar el tiempo de ejecución.

## Particionadores
Un particionador es una función que se utiliza para asignar claves a particiones. Los particionadores son útiles en operaciones de agrupación y ordenamiento, donde los datos se deben agrupar o ordenar según una clave específica.

Hay dos tipos de particionadores en Spark: el **particionador hash** y el **particionador de rango (o range)**.

El particionador hash es el particionador predeterminado en Spark. Este particionador utiliza una función de hash para asignar las claves a particiones. La función hash toma una clave y la convierte en un número entero. Luego, el particionador divide el rango de números enteros en un número determinado de particiones, y asigna las claves a las particiones basándose en el número de la función hash.

El particionador de rango, por otro lado, asigna las claves a las particiones basándose en el rango de valores de las claves. Por ejemplo, si se tiene un RDD de tuplas (k, v) y se desea ordenar el RDD por la clave k, el particionador de rango asignará las claves a las particiones basándose en el rango de valores de k. Cada partición tendrá un rango de valores de k específico, y las claves se asignarán a la partición correspondiente en función del rango.

El particionador de rango es especialmente útil cuando se trabaja con datos ordenados, ya que se puede garantizar que los datos con claves cercanas estarán en la misma partición. Esto puede mejorar el rendimiento de operaciones como la unión de RDD, la intersección y la operación de join.

En general, el particionador a utilizar dependerá del tipo de datos y la operación que se va a realizar. En muchos casos, el particionador predeterminado (hash) funciona bien, pero en otros casos, es necesario utilizar un particionador de rango personalizado para lograr el mejor rendimiento posible.

In [22]:
# Hash partitioner
rdd = sc.parallelize(["x", "y", "z"])
partitions = rdd.getNumPartitions()

In [23]:
# index = hash(key) % partitions
index_x = hash("x") % partitions
index_y = hash("y") % partitions
index_z = hash("z") % partitions

print(index_x, index_y, index_z)

1 11 5


# Shuffling en PySpark
El shuffling es una operación de procesamiento en Spark que implica la reorganización de los datos en un RDD (Resilient Distributed Dataset) a través de la red. El shuffling es una operación costosa, ya que requiere que los datos se muevan de un nodo de la red a otro, lo que puede llevar tiempo y consumir una gran cantidad de recursos de red y procesamiento.

En términos generales, el shuffling se utiliza cuando se necesita reorganizar los datos en un RDD de tal manera que los datos con la misma clave se encuentren en la misma partición. Por ejemplo, al realizar una operación de groupBy en un RDD, los datos deben agruparse por clave en cada partición y luego combinarse en una sola partición. Esto implica una operación de shuffling, ya que los datos deben ser reorganizados de tal manera que los datos con la misma clave se encuentren en la misma partición.

El proceso de shuffling en Spark implica tres fases principales:

Particionamiento: los datos del RDD se asignan a particiones en función de una clave.

Transferencia: los datos se transfieren de un nodo de la red a otro para asegurar que los datos con la misma clave estén en la misma partición.

Ordenamiento: los datos se ordenan dentro de cada partición en función de la clave.

El shuffling puede ser un proceso costoso, por lo que es importante minimizar su uso siempre que sea posible. Una forma de hacerlo es asegurarse de que los datos estén particionados de manera adecuada desde el principio, utilizando particionadores personalizados si es necesario. También se pueden utilizar operaciones como el reparticionamiento y la cache para optimizar el procesamiento y evitar el shuffling innecesario.

# Broadcast de variables en PySpark
El broadcasting de variables es una técnica utilizada en Spark para distribuir variables de solo lectura de manera eficiente a través de todos los nodos de un clúster de Spark. Esta técnica se utiliza para reducir el tráfico de red y mejorar el rendimiento del procesamiento de datos.

Cuando una variable se transmite mediante la técnica de broadcasting, se envía solo una vez desde el nodo controlador al resto de los nodos del clúster. Luego, la variable se almacena en caché en la memoria de los nodos receptores para que pueda ser utilizada en operaciones posteriores de manera local, sin necesidad de volver a enviar la variable a través de la red. Esto reduce significativamente el tráfico de red y mejora el rendimiento del procesamiento de datos, especialmente para variables grandes que se utilizan en múltiples operaciones.

Para utilizar la técnica de broadcasting de variables en Spark, se puede utilizar la función broadcast() que toma una variable de Python y devuelve una variable Broadcast de Spark que se puede utilizar en operaciones posteriores. La variable de broadcast se puede acceder en los nodos del clúster como si fuera una variable local, lo que permite una lectura eficiente y sin necesidad de transmitirla a través de la red.

Es importante tener en cuenta que las variables que se transmiten mediante la técnica de broadcasting deben ser de solo lectura, ya que no se pueden modificar en los nodos del clúster. Si se necesita modificar la variable, debe transmitirse de nuevo a través de la red.

In [26]:
rdd = sc.parallelize(range(10))

# Crear una variable broadcast
var1 = 1
var1_broadcast = sc.broadcast(var1)

In [27]:
# Acceder a la variable broadcast
rdd1 = rdd.map(lambda x: x + var1_broadcast.value)
rdd1.collect()

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [29]:
# Unpersistir la variable broadcast
var1_broadcast.unpersist()
# Aunque se libere la variable broadcast, los nodos del clúster podrán seguir accediendo a ella, pero se volverá a transmitir a través de la red.

1


In [31]:
# Destruir la variable broadcast
var1_broadcast.destroy()
rdd1 = rdd.map(lambda x: x + var1_broadcast.value)  # Error al acceder a la variable broadcast porque ya no existe
rdd1.collect()

Py4JJavaError: An error occurred while calling o112.destroy.
: org.apache.spark.SparkException: Attempted to use Broadcast(2) after it was destroyed (destroy at NativeMethodAccessorImpl.java:0) 
	at org.apache.spark.broadcast.Broadcast.assertValid(Broadcast.scala:144)
	at org.apache.spark.broadcast.Broadcast.destroy(Broadcast.scala:106)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(NativeMethodAccessorImpl.java:62)
	at sun.reflect.DelegatingMethodAccessorImpl.invoke(DelegatingMethodAccessorImpl.java:43)
	at java.lang.reflect.Method.invoke(Method.java:498)
	at py4j.reflection.MethodInvoker.invoke(MethodInvoker.java:244)
	at py4j.reflection.ReflectionEngine.invoke(ReflectionEngine.java:357)
	at py4j.Gateway.invoke(Gateway.java:282)
	at py4j.commands.AbstractCommand.invokeMethod(AbstractCommand.java:132)
	at py4j.commands.CallCommand.execute(CallCommand.java:79)
	at py4j.ClientServerConnection.waitForCommands(Cl

# Acumuladores en PySpark
Los acumuladores en Spark son variables que se pueden actualizar de manera segura y eficiente en paralelo durante la ejecución de tareas en un clúster. Son similares a las variables de broadcast, pero en lugar de ser sólo de lectura, se pueden agregar valores a ellas en cada tarea que se ejecuta en el clúster.

Los acumuladores son útiles para realizar tareas como recuento de elementos, sumas y promedios, entre otros. En lugar de crear una variable separada para cada tarea, se puede usar un acumulador para recolectar los valores de las tareas en un solo lugar y, al final de la ejecución, obtener el resultado deseado.

La actualización de los acumuladores se realiza de manera eficiente gracias a que Spark los actualiza de manera acumulativa en lugar de hacer una copia completa de la variable en cada tarea. Esto reduce la cantidad de datos que se deben transferir a través de la red y mejora el rendimiento de la aplicación.

Es importante tener en cuenta que los acumuladores sólo deben ser utilizados en operaciones de sólo escritura y que su comportamiento no es determinístico en operaciones de escritura y lectura. Por lo tanto, se recomienda utilizarlos únicamente en casos en los que se esté seguro de que se van a realizar operaciones de sólo escritura.

In [35]:
rdd = sc.parallelize([i for i in range(0, 11, 2)])
rdd.collect()

[0, 2, 4, 6, 8, 10]

In [37]:
# Crear un acumulador
accumulator = sc.accumulator(0)

In [40]:
# Utilizar el acumulador
rdd.foreach(lambda x: accumulator.add(x))
print(accumulator.value)  # Si se ejecuta varias veces, el valor del acumulador se incrementará en cada ejecución

90
